# Stop Detection

<img align="right" src="https://movingpandas.github.io/movingpandas/assets/img/movingpandas.png">

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/movingpandas/movingpandas/main?filepath=tutorials/4-stop-detection.ipynb)

**<p style="color:#e31883">This notebook demonstrates the current development version of MovingPandas.</p>**

For tutorials using the latest release visit https://github.com/movingpandas/movingpandas-examples.


## Setup

In [ ]:
import pandas as pd
import geopandas as gpd
from datetime import datetime, timedelta

import sys

sys.path.append("..")
import movingpandas as mpd

mpd.show_versions()

import warnings

warnings.simplefilter("ignore")

## Loading Geolife Sample

In [ ]:
%%time
gdf = gpd.read_file("data/demodata_geolife.gpkg")
print("Finished reading {} rows".format(len(gdf)))

In [ ]:
tc = mpd.TrajectoryCollection(gdf, "trajectory_id", t="t")
tc = mpd.MinTimeDeltaGeneralizer(tc).generalize(timedelta(seconds=10))
tc

## Stop Detection with a SingleTrajectory

There are no definitive answers when it comes to detecting / extracting stops from movement trajectories. Due to tracking inaccuracies, movement speed rarely goes to true zero. GPS tracks, for example, tend to keep moving around the object's stop location. 

Suitable stop definitions are also highly application dependent. For example, an application may be interested in analyzing trip purposes. To do so, analysts would be interested in stops that are longer than, for example, 5 minutes and may try to infer the purpose of the stop from the stop location and time. Shorter stops, such as delays at traffic lights, however would not be relevant for this appication. 

In the MovingPandas **TrajectoryStopDetector** implementation, a stop is detected if the movement stays within an area of specified size for at least the specified duration.

In [ ]:
my_traj = tc.trajectories[0]
my_traj

In [ ]:
detector = mpd.TrajectoryStopDetector(my_traj)

In [ ]:
hvplot_defaults = {
    "tiles": "CartoLight",
    "frame_height": 350,
    "frame_width": 350,
    "line_width": 7.0,
}
traj_plot = my_traj.hvplot(
    title="Trajectory {}".format(my_traj.id), color="slategray", **hvplot_defaults
)
traj_plot

### Stop duration

A sensible detectable stop duration should be a multiple of the trajectory sampling interval

In [ ]:
for traj in tc:
    print(traj.get_sampling_interval())

In [ ]:
%%time
stop_durations = detector.get_stop_time_ranges(
    min_duration=timedelta(seconds=60), max_diameter=100
)

In [ ]:
for x in stop_durations:
    print(x)

### Stop points

In [ ]:
%%time
stop_points = detector.get_stop_points(
    min_duration=timedelta(seconds=60), max_diameter=100
)

In [ ]:
stop_points

In [ ]:
stop_point_plot = traj_plot * stop_points.hvplot(
    geo=True, size="duration_s", color="deeppink"
)
stop_point_plot

### Stop segments

In [ ]:
%%time
stop_segments = detector.get_stop_segments(
    min_duration=timedelta(seconds=60), max_diameter=100
)

In [ ]:
stop_segments

In [ ]:
stop_segment_plot = stop_point_plot * stop_segments.hvplot(
    line_width=7.0, tiles=None, color="orange"
)
stop_segment_plot

### Split at stops

In [ ]:
%%time
split = mpd.StopSplitter(my_traj).split(
    min_duration=timedelta(seconds=60), max_diameter=100
)

In [ ]:
split

In [ ]:
for segment in split:
    print(segment)

In [ ]:
stop_segment_plot + split.hvplot(
    title="Trajectory {} split at stops".format(my_traj.id), **hvplot_defaults
)

## Stop Detection for TrajectoryCollections

The process is the same as for individual trajectories.

In [ ]:
%%time
detector = mpd.TrajectoryStopDetector(tc)
stops = detector.get_stop_segments(
    min_duration=timedelta(seconds=120), max_diameter=100
)
len(stops)

In [ ]:
traj_map = tc.hvplot(color="slategray", **hvplot_defaults)

In [ ]:
(
    traj_map
    * stops.hvplot(line_width=7.0, tiles=None, color="deeppink")
    * stops.get_start_locations().hvplot(geo=True, color="deeppink")
)

In [ ]:
stop_points = detector.get_stop_points(
    min_duration=timedelta(seconds=120), max_diameter=100
)

### Testing the impact of column data types

In [ ]:
map1 = traj_map * stop_points.hvplot(
    geo=True,
    size=200,
    color="traj_id",
    title=f"Traj_id type {type(stop_points.iloc[0].traj_id)}",
)

stop_points.traj_id = stop_points.traj_id.astype("str")
map2 = traj_map * stop_points.hvplot(
    geo=True,
    size=200,
    color="traj_id",
    title=f"Traj_id type {type(stop_points.iloc[0].traj_id)}",
)

map1 + map2

## Comparison of Detected Stop Events

In [ ]:
detector = mpd.TrajectoryStopDetector(tc)
stops1 = detector.get_stop_segments(min_duration=timedelta(seconds=120), max_diameter=100)
stops2 = detector.get_stop_segments(min_duration=timedelta(seconds=60), max_diameter=100)
print(stops1)
print(stops2)

Let's assume stops1 is the ground truth, and stops2 is the result we want to evaluate:

In [ ]:
groundtruth = stops1
prediction = stops2

#gt.hvplot(color='blue') * ev.hvplot(color='red')

In [ ]:
def events_overlap(event1, event2):
    start1, end1 = event1.get_start_time(), event1.get_end_time()
    start2, end2 = event2.get_start_time(), event2.get_end_time()
    i1 = pd.Interval(start1.timestamp(),end1.timestamp())
    i2 = pd.Interval(start2.timestamp(),end2.timestamp())
    return i1.overlaps(i2)

overlapping_gt_events = []
non_overlapping_gt_events = []
false_positives = []

# First pass: GT -> Pred
for gt_event in groundtruth.trajectories:
    gt_id = gt_event.parent.id
    gt_start = gt_event.get_start_time()
    gt_end = gt_event.get_end_time()

    is_overlapping = False

    for pred_event in prediction.trajectories:
        if pred_event.parent.id != gt_id:
            continue

        pred_start = pred_event.get_start_time()
        pred_end = pred_event.get_end_time()

        if events_overlap(gt_event, pred_event):
            is_overlapping = True
            break

    if is_overlapping:
        overlapping_gt_events.append((gt_id, gt_start, gt_end, pred_start, pred_end))
    else: 
        non_overlapping_gt_events.append((gt_id, gt_start, gt_end))

# Second pass: Pred -> GT (to find false positives)
for pred_event in prediction.trajectories:
    pred_id = pred_event.parent.id

    is_overlapping = False
    for gt_event in groundtruth.trajectories:
        if gt_event.parent.id != pred_id:
            continue

        if events_overlap(pred_event, gt_event):
            is_overlapping = True
            break

    if not is_overlapping:    
        pred_start = pred_event.get_start_time()
        pred_end = pred_event.get_end_time()
        false_positives.append((pred_id, pred_start, pred_end))

# Output results
print("Overlapping Events:")
for entry in overlapping_gt_events:
    print(f"Mover {entry[0]}: GT ({entry[1]} to {entry[2]}) overlaps with Pred ({entry[3]} to {entry[4]})")

print("\nNon-overlapping GT Events (False Negatives):")
for entry in non_overlapping_gt_events:
    print(f"Mover {entry[0]}: GT ({entry[1]} to {entry[2]}) has no overlapping prediction")

print("\nFalse Positives (Predicted Events with No Matching GT):")
for entry in false_positives:
    print(f"Mover {entry[0]}: Pred ({entry[1]} to {entry[2]}) has no overlapping ground truth")



In [ ]:
print(f"True positives: {len(groundtruth.trajectories)-len(non_overlapping_gt_events)} out of {len(groundtruth.trajectories)} ground truth events detected.")
print(f"False negatives: {len(non_overlapping_gt_events)} out of {len(groundtruth.trajectories)} ground truth events were not detected.")
print(f"False positives: {len(false_positives)} incorrect events detected. ")

## Continue exploring MovingPandas

1. [Getting started](1-getting-started.ipynb)
1. [Handling trajectory data files (reading & writing)](2-reading-data-from-files.ipynb)
1. [TrajectoryCollection aggregation (flow maps)](3-generalization-and-aggregation.ipynb)
1. [Stop detection](4-stop-detection.ipynb)
1. [Working with local coordinates](5-local-coordinates.ipynb)
1. [Computing trajectory metrics](6-trajectory-metrics.ipynb)
1. [Multithreading](7-multithreading.ipynb)
1. [OGC Moving Features](8-ogc-moving-features.ipynb)